In [1]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz

from pathlib import Path 
import time
import math

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/training_data_all_20220203.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103356 entries, 0 to 103355
Columns: 300 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(5), object(5)
memory usage: 236.6+ MB


In [3]:
usgs_reg = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/USGS_region_lookup.csv')
usgs_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NA_L3CODE   11 non-null     object
 1   USGS_Group  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [4]:
region_dict = dict(zip(usgs_reg.NA_L3CODE, usgs_reg.USGS_Group))
data['NewEco3'] = data['NewEco3'].replace(region_dict)
print(data['NewEco3'].unique())

['Xeric' 'Interior Mountains' 'North Coastal Mountains']


In [5]:
records_with_missing_data = len(data[data.isna().any(axis=1)])
print('Dropping {} records with missing data'.format(records_with_missing_data))
data = data.dropna(axis=0,how='any').copy()
#data.isnull().sum().sum()
data['flow_cfs'] = data['flow_cfs'] / data['DRAIN_SQKM']


Dropping 3 records with missing data


In [6]:
data.head(2)

,Unnamed: 0,gage_id,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
0,0,10258000,1950,1,0.010533,22593497,mean,observed,8.0,18.0,...,40.085585,36.469271,13.544983,6.353392,173.076519,164.956173,145.791419,48.690715,22.244474,19.426220
1,1,10258500,1950,1,0.001901,22593537,mean,observed,8.0,18.0,...,9.309282,18.295394,20.328231,8.441266,83.834380,83.283639,71.869830,6.863135,4.810008,4.094633


In [7]:
data['year'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021])

In [8]:
data = data.loc[(data['year'] <=2015) & (data['NewEco3'] == 'North Coastal Mountains')] 
data

,Unnamed: 0,gage_id,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
32,32,11160500,1950,1,0.747445,17682474,mean,observed,1.0,18.0,...,106.450111,231.959526,110.665901,65.782434,235.425883,228.491877,224.953219,54.051989,30.424252,28.584388
54,54,11372000,1950,1,0.440419,2495296,mean,observed,78.0,18.0,...,122.352341,245.873882,105.721910,39.411874,174.271086,167.375109,159.523532,47.588272,19.457454,17.080159
66,66,11475500,1950,1,5.369725,8287608,mean,observed,1.0,18.0,...,125.146984,266.709035,184.037859,109.181056,320.631575,308.257065,283.893160,100.495164,72.664160,69.502230
67,67,11476500,1950,1,3.880735,8285120,mean,observed,1.0,18.0,...,122.814190,261.179623,171.369155,101.530337,314.094604,305.616284,273.970399,96.411693,69.272203,65.781653
68,68,11477000,1950,1,2.422771,2704735,mean,observed,78.0,18.0,...,106.343313,205.158436,126.887635,74.335947,256.019972,249.002693,223.908904,77.377102,48.058846,44.735057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96597,95,11525500,2015,12,0.166211,8246426,mean,observed,78.0,18.0,...,110.085996,208.998279,133.792881,204.339891,108.635655,73.667683,59.087765,33.449802,6.256674,3.710198
96598,96,11525530,2015,12,0.540548,8246406,mean,observed,78.0,18.0,...,73.831520,155.103720,101.819523,179.342903,106.464020,65.587011,48.661061,23.299238,5.029743,3.024471
96599,97,11525670,2015,12,0.159632,8245912,mean,observed,78.0,18.0,...,65.346546,138.666745,82.778649,169.126236,121.336862,65.309628,45.556211,21.653712,4.847328,2.859299
96600,98,11528700,2015,12,0.795594,8232392,mean,observed,78.0,18.0,...,86.692378,176.082637,108.041588,188.779719,148.401944,117.056950,92.644146,30.419106,9.146426,6.518255


In [9]:
data['year'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [10]:
data['NewEco3'].unique()

array(['North Coastal Mountains'], dtype=object)

In [11]:
#dropping inessential columns
col = ['comid','Unnamed: 0','year','gage_id','statistic','variable','NHDPLUSREG','comidyear','ECO3','AggEcoRegion','NewEco3']
df = data.drop(columns = col)

In [12]:
df.head(2)

,month,flow_cfs,comid,DRAIN_SQKM,SLOPE_PCT_30M,ELEV_MEAN_M_BASIN_30M,ELEV_MIN_M_BASIN_30M,ELEV_MAX_M_BASIN_30M,ElvRng,TOT_ECOL3_1,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
32,1,0.747445,17682474,274.9149,32.96,365.61,69.57,984.49,914.92,100.0,...,106.450111,231.959526,110.665901,65.782434,235.425883,228.491877,224.953219,54.051989,30.424252,28.584388
54,1,0.440419,2495296,592.3971,40.48,856.09,206.55,1888.45,1681.90,0.0,...,122.352341,245.873882,105.721910,39.411874,174.271086,167.375109,159.523532,47.588272,19.457454,17.080159


In [15]:
def x_train(month):
  train = df[df['month']==month]
  train.reset_index(drop=True,inplace=True)
  x_train = train.drop(columns = ['flow_cfs']).copy()
  return x_train

def y_train(month):
  train = df[df['month']==month]
  train.reset_index(drop=True,inplace=True)
  y_train = train.flow_cfs.values 
  return y_train

def x_valid(month):
  valid = df[df['month']==month]
  valid.reset_index(drop=True,inplace=True)
  x_valid = valid.drop(columns = ['flow_cfs']).copy()
  return x_valid

def y_valid(month):
  valid = df[df['month']==month]
  valid.reset_index(drop=True,inplace=True)
  y_valid = valid.flow_cfs.values
  return y_valid

def x_test(month):  
  test = df[df['month']==month]
  test.reset_index(drop=True,inplace=True)
  x_test = test.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month):
  test = df[df['month']==month]
  test.reset_index(drop=True,inplace=True)  
  y_test = test.flow_cfs.values
  return y_test

In [16]:
def hyperparameter_csv_append(month, model_run_date_time, r2, leaf, features, depth, estimator):
  if exists('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Coastal_M-all-comids) model/hyperparameter_'+str(month)+'.csv') == False:

    with open('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Coastal_M-all-comids) model/hyperparameter_'+str(month)+'.csv', 'w') as f:
      writer = csv.writer(f)
      header = ["model_run_date_time", "R2", "leaf", "features", "depth", "estimator"]
      writer.writerow(header)
      f.close()

  with open('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Coastal_M-all-comids) model/hyperparameter_'+str(month)+'.csv', 'a') as f:
    row = [str(model_run_date_time), str(r2), str(leaf), str(features), str(depth), str(estimator)]
    new_row = ",".join(row)
    f.write(new_row)
    f.write("\n")

def hyper_parameter_search(n_iter, month): 
 
    for i in range(n_iter):   
        # n_estimators = The number of trees in the forest. 
        estimators = [1000] #ntrees in r. Ted used 1000

        for count in estimators:
            rf = RandomForestRegressor(n_jobs = -1, n_estimators=count, min_samples_leaf=1,max_features="sqrt", max_depth=5, verbose=1) 
            rf.fit(x_train(month), y_train(month))
            predictions = rf.predict(x_valid(month))
            features_sqrt = int(math.sqrt(len(x_train(month).columns)))

            # Mean absolute error (MAE)
            mae = mean_absolute_error(y_valid(month), predictions)
            # Mean squared error (MSE)
            mse = mean_squared_error(y_valid(month), predictions)
            # R-squared scores 
            r2 = r2_score(y_valid(month)+1, predictions+1)

            # Print metrics
            print('Mean Absolute Error:', round(mae, 2))
            print('Mean Squared Error:', round(mse, 2))
            print('R-squared scores:', round(r2, 2))            

            model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))

            hyperparameter_csv_append(month, model_run_date_time, r2, 1, features_sqrt, 5, count)  


In [19]:
def train_final_model(month):

  df_usgs_reduced = pd.read_excel('/content/drive/MyDrive/TNC_Collab_Results/USGS_reduced_variables(Zimmerman_etal_2017_Supp_info2).xlsx', sheet_name = 2) 
  #Change the sheetname according to region, Sheet 1 = Interior_Mountains, Sheet 2 = Coastal_Mountains, Sheet 3 = Xeric
  df_usgs_reduced = df_usgs_reduced.loc[(df_usgs_reduced['Monthly.Stat'] == 'Mean') & (df_usgs_reduced['Month'] == month)] 
  reduced_var_list = df_usgs_reduced['Predictor'].unique().tolist()
  print("Number of variables for final model: {}".format(len(reduced_var_list)))
  print(reduced_var_list)

  dfp = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Coastal_M-all-comids) model/hyperparameter_'+str(month)+'.csv')
  dfp = dfp.sort_values(by="R2", ascending=False)
  
  best_leaf = dfp['leaf'].iloc[0]
  best_features = dfp['features'].iloc[0]
  best_depth = dfp['depth'].iloc[0]
  best_estimator = dfp['estimator'].iloc[0]

  reduced_columns_list = ['month']
  reduced_columns_list = reduced_columns_list + reduced_var_list

  # Train Reduced model
  rf = RandomForestRegressor(n_jobs = -1, min_samples_leaf=best_leaf,max_features=best_features, max_depth=best_depth, n_estimators=best_estimator,verbose=1) 
  rf.fit(x_train(month).loc[:, reduced_columns_list], y_train(month))
  predictions = rf.predict(x_test(month).loc[:, reduced_columns_list])
  r2 = r2_score(y_test(month)+1, predictions+1)
  model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))
  print("R2 for USGS_Coastal Mountain_reduced model=",r2)

  rf_pickle_file = 'USGS_model_reduced_'+str(month)+'_r2_'+str(round(r2,4))+'_'+str(model_run_date_time)+'.pkl'
  with open('/content/drive/My Drive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Coastal_M-all-comids)/'+rf_pickle_file, 'wb') as f:
    cPickle.dump(rf, f)

In [20]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [11]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 11


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.38
Mean Squared Error: 0.4
R-squared scores: 0.74


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.38
Mean Squared Error: 0.4
R-squared scores: 0.74


...

Mean Absolute Error: 0.39
Mean Squared Error: 0.42
R-squared scores: 0.73


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.38
Mean Squared Error: 0.4
R-squared scores: 0.74
Training final model for month 11
Number of variables for final model: 49
['wb0', 'p0', 'p3sum', 'p2', 'p6sum', 'p2sum', 'p1', 'PET_CAT', 'PERMAVE', 'pmpe', 'CLAYAVE', 'p7', 'T_AVG_CAT', 'wb1', 'p11', 't3', 'wb3sum', 'wb2sum', 'RH_CAT', 'PPTAVG_BASIN', 't9', 't11', 'p6', 'WD_MAY_BASIN', 'p5', 'WD_MAR_BASIN', 'p4', 'ELEV_MIN_M_BASIN_30M', 'PET', 't1', 'CaO_pct', 'WD_SEP_BASIN', 'ROCKDEPAVE', 'ET', 'MgO_pct', 'WD_FEB_BASIN', 'RECHARGE', 'HLR20', 'HLR19', 'ANN_CONS_WETDAYS', 'WD_JAN_BASIN', 'WDMAX_BASIN', 'HGC', 'WD_BASIN', 'SiO_pct', 'WD_APR_BASIN', 'WD_NOV_BASIN', 'CAT_AnnMaxPrecip', 'SGEO21']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.9209558395068544


In [21]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [12]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 12


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.76
Mean Squared Error: 1.26
R-squared scores: 0.77


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.4s finished


Mean Absolute Error: 0.78
Mean Squared Error: 1.33
R-squared scores: 0.75


...

Mean Absolute Error: 0.78
Mean Squared Error: 1.3
R-squared scores: 0.76


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.77
Mean Squared Error: 1.28
R-squared scores: 0.76
Training final model for month 12
Number of variables for final model: 45
['wb0', 'p0', 'p1', 'wb1', 'p6sum', 'p3sum', 'pmpe', 'p2sum', 'PPTAVG_BASIN', 'CaO_pct', 'CLAYAVE', 'ROCKDEPAVE', 'p10', 'wb2sum', 'wb3sum', 'wb9', 'p3', 'p8', 't0', 'PET_CAT', 'p7', 't3', 'HLR19', 't7', 't5', 'p5', 'SiO_pct', 'RH_CAT', 'SILTAVE', 'SANDAVE', 'MgO_pct', 'HGD', 'T_AVG_CAT', 'PET', 'CAT_AnnMaxPrecip', 'WD_JAN_BASIN', 'WDMAX_BASIN', 'AnnMaxPrecip', 'WD_MAR_BASIN', 'WD_OCT_BASIN', 'WD_DEC_BASIN', 'PPTAVG_CAT', 'SGEO21', 'SGEO45', 'WD_APR_BASIN']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.9286399090314066


In [22]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [1]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.84
Mean Squared Error: 1.41
R-squared scores: 0.74


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.84
Mean Squared Error: 1.42
R-squared scores: 0.74


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.84
Mean Squared Error: 1.4
R-squared scores: 0.74
Training final model for month 1
Number of variables for final model: 49
['wb0', 'p0', 'p2sum', 'p6sum', 'wb1', 'p3sum', 'CaO_pct', 'p1', 'wb2sum', 'CLAYAVE', 'p10', 'wb6sum', 'wb3sum', 'ROCKDEPAVE', 't11', 't0', 'p2', 'p12', 'wb11', 'PET_CAT', 'p5', 'SILTAVE', 'pmpe', 'wb12', 't3', 'SiO_pct', 'SANDAVE', 'CONTACT', 'HLR19', 'HGD', 'ET', 'PERMAVE', 'PPTAVG_BASIN', 'ELEV_MIN_M_BASIN_30M', 'T_AVG_CAT', 'RH_CAT', 'WD_JAN_BASIN', 'PERHOR', 'CAT_AnnMaxPrecip', 'TOPWET', 'SGEO21', 'WD_OCT_BASIN', 'PPTAVG_CAT', 'WD_NOV_BASIN', 'SGEO45', 'HGA', 'HLR18', 'RFACT', 'WD_APR_BASIN']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s


R2 for USGS_Coastal Mountain_reduced model= 0.896403007679032


[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [23]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [2]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 2


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.82
Mean Squared Error: 1.27
R-squared scores: 0.69


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.82
Mean Squared Error: 1.26
R-squared scores: 0.69


...

Mean Absolute Error: 0.82
Mean Squared Error: 1.26
R-squared scores: 0.69


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.81
Mean Squared Error: 1.24
R-squared scores: 0.7
Training final model for month 2
Number of variables for final model: 43
['p0', 'wb0', 'pmpe', 'PPTAVG_BASIN', 'HLR19', 'p1', 'wb1', 'p5', 'p4', 'p2sum', 'wb4', 'p9', 't0', 'p6sum', 'p3sum', 'p8', 'p10', 'AnnMaxPrecip', 't1', 'CaO_pct', 'wb5', 'wb2sum', 't8', 't4', 'CLAYAVE', 'SGEO21', 'SILTAVE', 'SiO_pct', 'RFACT', 'HGD', 'CAT_AnnMaxPrecip', 'WD_APR_BASIN', 'PET_CAT', 'ROCKDEPAVE', 'SANDAVE', 'WD_JAN_BASIN', 'T_AVG_CAT', 'DEPTH_WATTAB', 'WD_OCT_BASIN', 'WD_MAR_BASIN', 'HGA', 'SGEO45', 'PPTAVG_CAT']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8907040903544466


In [24]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [3]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 3


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.61
Mean Squared Error: 0.72
R-squared scores: 0.72


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.62
Mean Squared Error: 0.73
R-squared scores: 0.72


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.62
Mean Squared Error: 0.73
R-squared scores: 0.72
Training final model for month 3
Number of variables for final model: 40
['p0', 'wb0', 'pmpe', 'PPTAVG_BASIN', 'HLR19', 'wb1', 'p1', 'p11', 'p2sum', 'p6sum', 'wb11', 'p3sum', 'WD_APR_BASIN', 'wb2sum', 'p6', 'p10', 'RECHARGE', 'p9', 'p4', 't9', 'wb3sum', 'wb6sum', 'CaO_pct', 'CAT_AnnMaxPrecip', 'WD_FEB_BASIN', 'CLAYAVE', 'WD_MAR_BASIN', 'HGD', 'PPTAVG_CAT', 'WD_BASIN', 'WDMAX_BASIN', 'SiO_pct', 'AnnMaxPrecip', 'ROCKDEPAVE', 'WD_DEC_BASIN', 'SGEO21', 'WD_JAN_BASIN', 'RFACT', 'SGEO45', 'HLR18']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s


R2 for USGS_Coastal Mountain_reduced model= 0.8847153194839996


[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [25]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [4]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 4


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.42
Mean Squared Error: 0.34
R-squared scores: 0.72


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.42
Mean Squared Error: 0.34
R-squared scores: 0.73


...

Mean Absolute Error: 0.42
Mean Squared Error: 0.34
R-squared scores: 0.73


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.42
Mean Squared Error: 0.34
R-squared scores: 0.73
Training final model for month 4
Number of variables for final model: 46
['p0', 'wb0', 'p1', 'p6sum', 'wb1', 'wb6sum', 'p6', 'WDMIN_BASIN', 'WD_JUL_BASIN', 'AnnMinPrecip', 'wb6', 'wb4', 'RECHARGE', 'p2sum', 'ET', 'WD_MAY_BASIN', 'wb5', 't0', 't4', 'p3sum', 'wb2sum', 't2', 'p8', 'p5', 'wb3sum', 'WD_SEP_BASIN', 'T_MAX_BASIN', 't1', 'p7', 'p3', 't5', 'MgO_pct', 't3', 'T_AVG_BASIN', 'ANN_CONS_WETDAYS', 'FST32F_BASIN', 'KO_pct', 'WD_APR_BASIN', 'WD_AUG_BASIN', 'NO10AVE', 'UCS', 'WD_JUN_BASIN', 'T_MIN_BASIN', 'BDAVE', 'NO4AVE', 'PERHOR']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.850077393791362


In [26]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [5]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 5


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.15
R-squared scores: 0.76


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.15
R-squared scores: 0.76


...

Mean Absolute Error: 0.26
Mean Squared Error: 0.15
R-squared scores: 0.76


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.15
R-squared scores: 0.76
Training final model for month 5
Number of variables for final model: 47
['wb0', 'WDMIN_BASIN', 'ET', 'WD_JUL_BASIN', 'p0', 'wb1', 'AnnMinPrecip', 'T_MAX_BASIN', 'p2sum', 't1', 'p1', 'p6sum', 't2', 'wb2sum', 'WD_APR_BASIN', 'ultramafic', 'wb3sum', 't0', 't12', 'WD_AUG_BASIN', 'p2', 'p11', 'p3sum', 't11', 'MgO_pct', 'RECHARGE', 'sedimentary', 'ELEV_MEAN_M_BASIN_30M', 'WD_SEP_BASIN', 'ANN_CONS_WETDAYS', 'ELEV_MAX_M_BASIN_30M', 'FeO_pct', 'WD_MAY_BASIN', 'KFACT_UP', 'WD_JUN_BASIN', 'T_MIN_BASIN', 'SLOPE_PCT_30M', 'FST32F_BASIN', 'ElvRng', 'SGEO8', 'PERHOR', 'NO200AVE', 'CLAYAVE', 'UCS', 'LST32F_BASIN', 'CAT_AnnMinPrecip', 'P_pct']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8408841908945036


In [27]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [6]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 6


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.05
R-squared scores: 0.78


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.05
R-squared scores: 0.78


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.05
R-squared scores: 0.78
Training final model for month 6
Number of variables for final model: 47
['ET', 'WDMIN_BASIN', 'WD_JUL_BASIN', 'p1', 'wb0', 'p0', 'T_MAX_BASIN', 'AnnMinPrecip', 'p2sum', 'wb1', 't1', 'p3sum', 'ELEV_MAX_M_BASIN_30M', 'wb2sum', 'wb3sum', 'p6sum', 'ultramafic', 'wb6sum', 't12', 'ELEV_MEAN_M_BASIN_30M', 'wb2', 't3', 'wb3', 'MgO_pct', 'p3', 't0', 't11', 't9', 'WD_SEP_BASIN', 'p2', 't2', 'WD_AUG_BASIN', 'KFACT_UP', 'FST32F_BASIN', 'FeO_pct', 'sedimentary', 'RECHARGE', 'BFI_AVE', 'ElvRng', 'T_MIN_BASIN', 'WD_JUN_BASIN', 'PERHOR', 'HLR20', 'NO200AVE', 'SLOPE_PCT_30M', 'NO10AVE', 'UCS']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s


R2 for USGS_Coastal Mountain_reduced model= 0.8564779827425147


[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [28]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [7]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.81


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.81


...

Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.81


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.81
Training final model for month 7
Number of variables for final model: 49
['ET', 'AnnMinPrecip', 'WDMIN_BASIN', 'WD_JUL_BASIN', 'wb1', 'wb0', 'p3sum', 'wb2sum', 'T_MAX_BASIN', 'p2sum', 'wb3sum', 'wb2', 'WD_AUG_BASIN', 'wb6sum', 't2', 't11', 'p1', 't1', 'wb3', 't0', 'p6sum', 'wb4', 'p4', 'FeO_pct', 'p0', 'MgO_pct', 'KO_pct', 't10', 'PET', 'BFI_AVE', 'KFACT_UP', 'RECHARGE', 'WD_SEP_BASIN', 'ELEV_MEAN_M_BASIN_30M', 'sedimentary', 'ultramafic', 'T_AVG_BASIN', 'UCS', 'ELEV_MAX_M_BASIN_30M', 'WD_JUN_BASIN', 'ElvRng', 'T_MIN_BASIN', 'SLOPE_PCT_30M', 'CLAYAVE', 'SGEO8', 'FST32F_BASIN', 'LST32F_BASIN', 'CAT_AnnMinPrecip', 'ANN_CONS_WETDAYS']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8585659206071813


In [29]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [8]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 8


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.81


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.81


...

Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.81


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.81
Training final model for month 8
Number of variables for final model: 58
['AnnMinPrecip', 'ET', 'WD_JUL_BASIN', 'WDMIN_BASIN', 'wb0', 'WD_AUG_BASIN', 'wb6sum', 'p0', 'BFI_AVE', 'T_MAX_BASIN', 'wb4', 'p6sum', 'WD_JUN_BASIN', 'p3sum', 'wb5', 'wb3sum', 'p4', 'wb2', 'wb2sum', 'wb3', 'wb1', 'sedimentary', 'ultramafic', 'FeO_pct', 'p5', 'PERMAVE', 'CAT_AnnMinPrecip', 'SLOPE_PCT_30M', 'HGA', 'UCS', 'WD_APR_BASIN', 'KO_pct', 'PERHOR', 'WD_SEP_BASIN', 'HGC', 'KFACT_UP', 'SGEO45', 'MgO_pct', 'CLAYAVE', 'SGEO21', 'TOT_ECOL3_1', 'HGB', 'RECHARGE', 'RFACT', 'ELEV_MEAN_M_BASIN_30M', 'WD_MAR_BASIN', 'WD_BASIN', 'LST32F_BASIN', 'BDAVE', 'FST32F_BASIN', 'TOPWET', 'Lperm', 'ANN_CONS_WETDAYS', 'ElvRng', 'ELEV_MAX_M_BASIN_30M', 'P_pct', 'SGEO8', 'volcanic']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.841765384175834


In [30]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [9]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 9


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.71


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.71


...

Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.71


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.71
Training final model for month 9
Number of variables for final model: 65
['p0', 'wb0', 'AnnMinPrecip', 'CAT_AnnMinPrecip', 't7', 'p5', 'ET', 'WD_JUL_BASIN', 'WDMIN_BASIN', 'p2sum', 'p12', 'sedimentary', 't9', 'WD_JUN_BASIN', 't5', 'wb1', 'T_AVG_CAT', 'wb12', 'p8', 'wb7', 'wb5', 'BFI_AVE', 'HLR9', 'p11', 't2', 'wb2sum', 'HLR12', 'p1', 'p6sum', 'p3sum', 't6', 't8', 't10', 'P_pct', 'wb3', 'p6', 't0', 'HGD', 'wb6', 'FST32F_BASIN', 'PERMAVE', 'SLOPE_PCT_30M', 'ultramafic', 'WD_BASIN', 'Lperm', 'WD_MAY_BASIN', 'T_MAX_BASIN', 'HGA', 'RECHARGE', 'PERHOR', 'wb11', 'KFACT_UP', 'WD_AUG_BASIN', 'PPTAVG_BASIN', 'RH_BASIN', 'ANN_CONS_WETDAYS', 'S_pct', 'UCS', 'WD_SEP_BASIN', 'LST32F_BASIN', 'ELEV_MAX_M_BASIN_30M', 'KO_pct', 'HGB', 'MgO_pct', 'FeO_pct']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8238742238291791


In [31]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [10]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 10


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.03
R-squared scores: 0.7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.03
R-squared scores: 0.7


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.03
R-squared scores: 0.7
Training final model for month 10
Number of variables for final model: 66
['wb0', 'p0', 'p2', 'p9', 't9', 'p3sum', 'p2sum', 'p1', 'WD_MAR_BASIN', 't12', 'p4', 'p6', 'WD_MAY_BASIN', 'RH_CAT', 'WD_JUN_BASIN', 'AnnMinPrecip', 'ELEV_MIN_M_BASIN_30M', 't2', 'WD_AUG_BASIN', 'wb1', 't11', 'RECHARGE', 'PET_CAT', 'wb5', 'wb9', 'WD_SEP_BASIN', 'p5', 'wb4', 'p11', 'CAT_AnnMinPrecip', 'wb6', 't5', 'p10', 'p3', 'wb6sum', 'wb7', 't6', 'S_pct', 'ROCKDEPAVE', 'wb3', 'OMAVE', 't10', 'SLOPE_PCT_30M', 't3', 'CaO_pct', 't0', 'Lperm', 'HLR20', 'KO_pct', 'NO4AVE', 'ANN_CONS_WETDAYS', 'WDMAX_BASIN', 'SiO_pct', 'PERDUN', 'P_pct', 'MgO_pct', 'WD_BASIN', 'WD_FEB_BASIN', 'FST32F_BASIN', 'HGD', 'TOPWET', 'T_MIN_BASIN', 'PPTAVG_CAT', 'HLR12', 'WD_JAN_BASIN', 'CAT_AnnMaxPrecip']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s


R2 for USGS_Coastal Mountain_reduced model= 0.8611885644611368


[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished
